## 4.1 데이터에서 학습한다!

신경망은 데이터에서 학습한다. 

여기서 학습이란, 신경망의 최적의 가중치 매개변수를 찾는 것이라고 말할 수 있다.

## 4.2 손실함수

신경망이 학습할 때, 현재의 상태를 하나의 지표로 표현하는데, 그것이 손실함수

보통 오차제곱합 과 교차 엔트로피 오차를 사용한다.

* 오차 제곱합

신경망의 출력에서 실제 정답 레이블을 뺀 것의 제곱을 모두 합산해 평균낸 것.

* 교차 엔트로피 오차

정답일 때의 추정의 자연로그를 계산하는 식

* 미니 배치 학습

모든 훈련 데이터를 대상으로 손실함수의 값을 구하는 것은 불가능하므로, 우리는 미니배치라는 훈련 데이터로부터 일부만 무작위로 고른 데이터의 묶음을 골라 학습을 수행한다.

## 4.3 수치 미분

미분이란 기울기, 즉 x의 작은 변화가 함수f(x)를 얼마나 변화시키는지를 나타낸 변화량을 의미.

* 편미분

변수가 여러개인 함수에 대해 각각에 매개변수에 대한 미분을 구하는 것.

-> 특정 장소에 대한 기울기를 구하는 것!

목표 변수 하나에 초점을 맞추고 다른 변수는 값ㅇ르 고정한다.

## 4.4 기울기

In [4]:
import numpy as np
import matplotlib.pylab as plt

In [5]:
def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)  # x와 형상이 같은 배열을 생성

    for idx in range(x.size):
        tmp_val = x[idx]
        # f(x+h) 계산
        x[idx] = tmp_val + h
        fxh1 = f(x)

        # f(x-h) 계산
        x[idx] = tmp_val - h
        fxh2 = f(x)

        grad[idx] = (fxh1 - fxh2) / (2 * h)
        x[idx] = tmp_val  # 값 복원

    return grad

In [6]:
def function_2(x):
    return x[0]**2 + x[1]**2
    # or return np.sum(x**2)


print(numerical_gradient(function_2, np.array([3.0, 4.0])))  # [ 6.  8.]
print(numerical_gradient(function_2, np.array([0.0, 2.0])))  # [ 0.  4.]
print(numerical_gradient(function_2, np.array([3.0, 0.0])))  # [ 6.  0.]

[6. 8.]
[0. 4.]
[6. 0.]


In [7]:
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    x_history = []

    for i in range(step_num):
        x_history.append(x.copy())
        grad = numerical_gradient(f, x)
        x -= lr * grad

    return x, np.array(x_history)

In [8]:
init_x = np.array([-3.0, 4.0])
x, x_history = gradient_descent(function_2, init_x, lr=0.1)
print(x)  # [ -6.11110793e-10   8.14814391e-10]

[-6.11110793e-10  8.14814391e-10]


In [9]:
init_x = np.array([-3.0, 4.0])
x, x_history = gradient_descent(function_2, init_x, lr=10.0)
print(x)  # [ -2.58983747e+13  -1.29524862e+12] 발산함

[-2.58983747e+13 -1.29524862e+12]


In [10]:
init_x = np.array([-3.0, 4.0])
x, x_history = gradient_descent(function_2, init_x, lr=1e-10)
print(x)  # [-2.99999994  3.99999992] 거의 변화 없음

[-2.99999994  3.99999992]


## 4.5 학습 알고리즘 구현하기

학습이란?

-> 신경망의 최적의 매개변수(가중치, 편향)를 찾아나가는 과정

신경망의 학습 단계

1. 미니배치(훈련 데이터의 일부를 무작위로 가져옴(미니배치) 미니배치의 손실함수를 줄이는 것이 목표

2. 기울기 산출
미니배치의 손실함수를 줄이기 위해, 각 가중치 매개변수의 기울기를 구한다. 기울기는 손실함수의 값을 낮추는 방향을 제시한다.

3. 매개변수 갱신
가중치 매개변수를 기울기 방향으로 조금 갱신

4. 반복
1~3 단계를 반복한다.

In [13]:
import sys, os
sys.path.append("C:/Users/kyw97/Desktop/CUAI/딥러닝/deep-learning-from-scratch-master")

In [15]:
from common.functions import *
from common.gradient import numerical_gradient

In [19]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size,
                 weight_init_std = 0.01):
        self.params={}
        self.params['W1'] = weight_init_std*np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std*np.random.randn(input_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']

        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)

        return y
    
    def loss(self, x, t):
        y = self.predict(x)

        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

## 4.5.2 미니배치 학습 구현하기

In [23]:
import numpy as np
import sys
import os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist
#from two_layer_net import TwoLayerNet


# 4.5.2 미니배치 학습 구현하기
# * 주의 : 아주 오래 걸림 *
"""
60000개의 훈련 데이터에서 임의로 100개의 데이터(이미지&정답 레이블)을 추려냄.
100개의 미니배치를 대상으로 확률적 경사 하강법을 수행해 매개변수를 갱신한다.
경사법에 의한 갱신 횟수를 1000번으로 설정하고 갱신할 때마다 손실 함수를 계산한다.
"""
(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=False)

# 하이퍼 파라메터
iters_num = 1000  # 반복횟수
train_size = x_train.shape[0]
batch_size = 100  # 미니배치 크기
learning_rate = 0.1
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    print(i)
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)
    # grad = network.gradient(x_batch, t_batch)  # 다음 장에서 구현할 더 빠른 방법!

    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

0


ValueError: shapes (100,50) and (784,10) not aligned: 50 (dim 1) != 784 (dim 0)

In [26]:
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

# 하이퍼파라미터
iters_num = 10000  # 반복 횟수를 적절히 설정한다.
train_size = x_train.shape[0]
batch_size = 100   # 미니배치 크기
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)
    #grad = network.gradient(x_batch, t_batch)
    
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

ValueError: shapes (100,50) and (784,10) not aligned: 50 (dim 1) != 784 (dim 0)

### 4.5.3 시험 데이터로 평가하기

1에폭마다 모든 훈련데이터와 시험데이터에 대한 정확도를 계산 -> for 문안에서 매번 계산하기에는 시간이 오래걸리고 자주 기록할 필요가 없음.

-> 에폭이 진행될수록 정확도가 모두 좋아짐 

-> 오버피팅이 일어나지 않았다.